In [204]:
# # Example usage
# images_dir = "/Users/ankushdhawan/Documents/Stanford/Pupper/tactile_gait/pupper_tactile_gait/classification/data_processing/dataset/0 degrees_new/0-felt_pad/images"
# flow_output_dir = "/Users/ankushdhawan/Documents/Stanford/Pupper/tactile_gait/pupper_tactile_gait/classification/data_processing/dataset/0 degrees_new/0-felt_pad/flow_visualizations"
# csv_path = "/Users/ankushdhawan/Documents/Stanford/Pupper/tactile_gait/pupper_tactile_gait/classification/data_processing/dataset/0 degrees_new/0-felt_pad/contact.csv"

In [205]:
# import os
# import csv
# import re
# from collections import defaultdict

# def process_trials(images_dir, csv_path):
#     """
#     Process images and contact data to identify trials.
    
#     A trial is defined as starting from out of contact (0) and then moving into contact (1).
    
#     Args:
#         images_dir (str): Path to directory containing images
#         csv_path (str): Path to CSV file containing contact data
        
#     Returns:
#         dict: Dictionary mapping trial numbers to lists of image filenames
#         int: Total number of trials
#     """
#     # Read contact data from CSV
#     contact_data = {}
#     with open(csv_path, 'r') as csv_file:
#         reader = csv.reader(csv_file)
#         headers = next(reader)  # Skip header row
        
#         # Find timestamp and contact columns
#         timestamp_col = None
#         contact_col = None
#         for i, header in enumerate(headers):
#             if 'time' in header.lower():
#                 timestamp_col = i
#             if 'contact' in header.lower():
#                 contact_col = i
        
#         if timestamp_col is None or contact_col is None:
#             raise ValueError("CSV file must contain timestamp and contact columns")
        
#         # Read data
#         for row in reader:
#             timestamp = row[timestamp_col]
#             contact = int(row[contact_col])
#             contact_data[timestamp] = contact
    
#     # Get all jpg images
#     image_files = [f for f in os.listdir(images_dir) if f.lower().endswith('.jpg')]
    
#     # Extract timestamps from image filenames
#     image_timestamps = {}
#     for img_file in image_files:
#         match = re.search(r'img_(\d+(?:\.\d+)?)', img_file)
#         if match:
#             timestamp = match.group(1)
#             image_timestamps[img_file] = timestamp
    
#     # Associate images with contact data
#     image_contact = {}
#     for img_file, timestamp in image_timestamps.items():
#         if timestamp in contact_data:
#             image_contact[img_file] = contact_data[timestamp]
#         else:
#             # Try to find closest timestamp
#             closest_ts = min(contact_data.keys(), key=lambda x: abs(float(x) - float(timestamp)))
#             image_contact[img_file] = contact_data[closest_ts]
    
#     # Sort images by timestamp
#     sorted_images = sorted(image_timestamps.keys(), key=lambda x: float(image_timestamps[x]))
    
#     # Group images into trials
#     trials = defaultdict(list)
#     trial_num = 0
#     in_trial = False
    
#     for img in sorted_images:
#         contact = image_contact[img]
        
#         # Start of a new trial: transition from no contact to contact
#         if not in_trial and contact == 0:
#             # We've found the beginning of a potential trial (no contact)
#             in_trial = True
#             trial_num += 1
#             trials[trial_num].append(img)
#         elif in_trial:
#             trials[trial_num].append(img)
            
#             # If we see contact, mark the end of the beginning phase
#             if contact == 1:
#                 # We've entered the contact phase, so we're done with this trial
#                 in_trial = False
    
#     # Remove incomplete trials (trials that didn't transition to contact)
#     complete_trials = {}
#     for trial_id, images in trials.items():
#         # Check if any image in this trial has contact=1
#         has_contact = any(image_contact[img] == 1 for img in images)
#         if has_contact:
#             complete_trials[trial_id] = images
    
#     return complete_trials, len(complete_trials)

# trials, num_trials = process_trials(images_dir, csv_path)

# print(f"Trials:", trials)

# print(f"Total number of trials: {num_trials}")

# # Print details of first few trials
# for trial_id in list(trials.keys())[:3]:
#     print(f"\nTrial {trial_id}:")
#     print(f"  Number of images: {len(trials[trial_id])}")
#     print(f"  First few images: {trials[trial_id][:3]}")


In [206]:
# Optical Flow

In [ ]:
# Set paths
# Example usage
object_nm = "0-foam"
folder_path = "0 degrees_new"
images_dir = f"/Users/ankushdhawan/Documents/Stanford/Pupper/tactile_gait/pupper_tactile_gait/classification/data_processing/dataset/{folder_path}/{object_nm}/images"
csv_path = f"/Users/ankushdhawan/Documents/Stanford/Pupper/tactile_gait/pupper_tactile_gait/classification/data_processing/dataset/{folder_path}/{object_nm}/contact.csv"
flow_npz_path = f"/Users/ankushdhawan/Documents/Stanford/Pupper/tactile_gait/pupper_tactile_gait/classification/data_processing/dataset/{folder_path}/{object_nm}/npz_files/{object_nm}_flow_data.npz"

In [208]:
import os
import csv
import re
import cv2
import numpy as np
from collections import defaultdict

def process_trials_with_optical_flow(images_dir, csv_path, save_npz=True, npz_output_path=None):
    """
    Process images and contact data to identify trials, then compute dense optical flow for each trial.
    
    Args:
        images_dir (str): Path to directory containing images
        csv_path (str): Path to CSV file containing contact data
        save_npz (bool): Whether to save flow data to NPZ file
        npz_output_path (str): Path where to save the NPZ file
        
    Returns:
        dict: Dictionary mapping trial numbers to optical flow data
        int: Total number of trials
    """
    # First, process trials using the function we already developed
    trials, num_trials = process_trials(images_dir, csv_path)
    
    # Process each trial with the dense optical flow function
    trial_flows = {}
    for trial_id, image_files in trials.items():
        # Get full paths to the images
        image_paths = [os.path.join(images_dir, img) for img in image_files]
        # Compute optical flow for this trial
        flow_data = denseOpticalFlowFromImages(image_paths)
        trial_flows[trial_id] = flow_data
    
    # Save the flow data to NPZ file if requested
    if save_npz:
        save_flow_data_to_npz(trial_flows, trials, npz_output_path)
        
    return trial_flows, num_trials

def save_flow_data_to_npz(trial_flows, trials, output_path=None):
    """
    Save optical flow data to NPZ file.
    
    Args:
        trial_flows (dict): Dictionary mapping trial IDs to optical flow data
        trials (dict): Dictionary mapping trial IDs to lists of image filenames
        output_path (str): Path to save the NPZ file. If None, saves as "flow_data.npz"
    
    Returns:
        str: Path to the saved NPZ file
    """
    if output_path is None:
        # Default output path
        output_path = "flow_data.npz"
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Dictionary to hold all data for NPZ file
    npz_data = {}
    
    # Add trial IDs
    trial_ids = np.array(list(trial_flows.keys()))
    npz_data['trial_ids'] = trial_ids
    
    # Process each trial
    for trial_id, flow_data in trial_flows.items():
        if flow_data is not None:
            # Add flow data for this trial
            npz_data[f'trial_{trial_id}_flow'] = flow_data
            
            # Add metadata
            npz_data[f'trial_{trial_id}_filenames'] = np.array(trials[trial_id], dtype=object)
            npz_data[f'trial_{trial_id}_shape'] = np.array(flow_data.shape)
    
    # Save to NPZ file
    np.savez_compressed(output_path, **npz_data)
    print(f"Saved optical flow data to {output_path}")
    
    return output_path

def denseOpticalFlowFromImages(image_paths, num_channels=2):
    """
    Compute dense optical flow from a sequence of image files.
    
    Args:
        image_paths (list): List of paths to image files in sequence order
        num_channels (int): Number of channels in the output (2 for magnitude and angle)
        
    Returns:
        numpy.ndarray: Pooled optical flow values with shape (num_channels, height//6, width//6)
    """
    # Load all frames
    saved_frames = []
    for img_path in image_paths:
        frame = cv2.imread(img_path)
        if frame is not None:
            saved_frames.append(frame)
    
    total_num_frames = len(saved_frames)
    
    if total_num_frames < 2:  # Need at least 2 frames for optical flow
        print(f"Insufficient frames found ({total_num_frames}), need at least 2")
        return None
    
    # Get first frame in grayscale
    old_gray = cv2.cvtColor(saved_frames[0], cv2.COLOR_BGR2GRAY)
    height, width = old_gray.shape
    
    # Initialize cumulative flow
    total_mag = np.zeros((height, width), dtype=np.float32)
    total_ang = np.zeros((height, width), dtype=np.float32)
    frame_count = 0
    
    # Process each frame
    for i in range(1, total_num_frames):
        frame = saved_frames[i]
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculates dense optical flow by Farneback method
        flow = cv2.calcOpticalFlowFarneback(
            old_gray, frame_gray,
            None,
            0.5, 2, 8, 2, 5, 1.1, 0
        )
        
        # Computes the magnitude and angle of the 2D vectors
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Accumulate flow
        total_mag += mag
        total_ang += ang
        frame_count += 1
        
        old_gray = frame_gray.copy()
    
    # Get average flow
    if frame_count > 0:
        avg_mag = total_mag / frame_count
        avg_ang = total_ang / frame_count
    else:
        avg_mag = total_mag
        avg_ang = total_ang
    
    # Apply spatial pooling (6x6 neighborhood)
    pooled_height = height // 6
    pooled_width = width // 6
    pooled_mag = np.zeros((pooled_height, pooled_width), dtype=np.float32)
    pooled_ang = np.zeros((pooled_height, pooled_width), dtype=np.float32)
    
    for j in range(pooled_height):
        for k in range(pooled_width):
            y_start = j * 6
            y_end = min((j+1) * 6, height)
            x_start = k * 6
            x_end = min((k+1) * 6, width)
            
            pooled_mag[j, k] = np.mean(avg_mag[y_start:y_end, x_start:x_end])
            pooled_ang[j, k] = np.mean(avg_ang[y_start:y_end, x_start:x_end])
    
    # Return pooled flow values with shape (2, height//6, width//6)
    pooled_flow = np.zeros((num_channels, pooled_height, pooled_width))
    pooled_flow[0] = pooled_mag
    pooled_flow[1] = pooled_ang
    
    return pooled_flow

def process_trials(images_dir, csv_path):
    """
    Process images and contact data to identify trials.
    
    A trial is defined as starting from out of contact (0) and then moving into contact (1).
    
    Args:
        images_dir (str): Path to directory containing images
        csv_path (str): Path to CSV file containing contact data
        
    Returns:
        dict: Dictionary mapping trial numbers to lists of image filenames
        int: Total number of trials
    """
    # Read contact data from CSV
    contact_data = {}
    with open(csv_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        headers = next(reader)  # Skip header row
        
        # Find timestamp and contact columns
        timestamp_col = None
        contact_col = None
        for i, header in enumerate(headers):
            if 'time' in header.lower():
                timestamp_col = i
            if 'contact' in header.lower():
                contact_col = i
        
        if timestamp_col is None or contact_col is None:
            raise ValueError("CSV file must contain timestamp and contact columns")
        
        # Read data
        for row in reader:
            timestamp = row[timestamp_col]
            contact = int(row[contact_col])
            contact_data[timestamp] = contact
    
    # Get all jpg images
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith('.jpg')]
    
    # Extract timestamps from image filenames
    image_timestamps = {}
    for img_file in image_files:
        match = re.search(r'img_(\d+(?:\.\d+)?)', img_file)
        if match:
            timestamp = match.group(1)
            image_timestamps[img_file] = timestamp
    
    # Associate images with contact data
    image_contact = {}
    for img_file, timestamp in image_timestamps.items():
        if timestamp in contact_data:
            image_contact[img_file] = contact_data[timestamp]
        else:
            # Try to find closest timestamp
            closest_ts = min(contact_data.keys(), key=lambda x: abs(float(x) - float(timestamp)))
            image_contact[img_file] = contact_data[closest_ts]
    
    # Sort images by timestamp
    sorted_images = sorted(image_timestamps.keys(), key=lambda x: float(image_timestamps[x]))
    
    # Group images into trials
    trials = defaultdict(list)
    trial_num = 0
    in_trial = False
    
    for img in sorted_images:
        contact = image_contact[img]
        
        # Start of a new trial: transition from no contact to contact
        if not in_trial and contact == 0:
            # We've found the beginning of a potential trial (no contact)
            in_trial = True
            trial_num += 1
            trials[trial_num].append(img)
        elif in_trial:
            trials[trial_num].append(img)
            
            # If we see contact, mark the end of the beginning phase
            if contact == 1:
                # We've entered the contact phase, so we're done with this trial
                in_trial = False
    
    # Remove incomplete trials (trials that didn't transition to contact)
    complete_trials = {}
    for trial_id, images in trials.items():
        # Check if any image in this trial has contact=1
        has_contact = any(image_contact[img] == 1 for img in images)
        if has_contact:
            complete_trials[trial_id] = images
    
    return complete_trials, len(complete_trials)

# Example usage
import csv
import re

# Process trials and compute optical flow, saving to NPZ
trial_flows, num_trials = process_trials_with_optical_flow(
    images_dir, csv_path, save_npz=True, npz_output_path=flow_npz_path
)

print(f"Total number of trials: {num_trials}")

# Print details of the optical flow data for the first few trials
for trial_id in list(trial_flows.keys())[:3]:
    flow_data = trial_flows[trial_id]
    if flow_data is not None:
        print(f"\nTrial {trial_id}:")
        print(f"  Optical flow shape: {flow_data.shape}")
        print(f"  Mean magnitude: {np.mean(flow_data[0])}")
        print(f"  Mean angle: {np.mean(flow_data[1])}")

# Example of how to load and use the flow NPZ file
def load_flow_data(npz_path):
    """Load optical flow data from NPZ file."""
    data = np.load(npz_path, allow_pickle=True)
    
    # Print available arrays in the NPZ file
    print(f"Available data in the NPZ file: {list(data.keys())}")
    
    # Get trial IDs
    trial_ids = data['trial_ids']
    print(f"Trial IDs: {trial_ids}")
    
    # Example: Access flow data for the first trial
    if len(trial_ids) > 0:
        first_trial_id = trial_ids[0]
        print(f"\nFirst trial (ID: {first_trial_id}):")
        print(f"  Flow data shape: {data[f'trial_{first_trial_id}_flow'].shape}")
        print(f"  Mean magnitude: {np.mean(data[f'trial_{first_trial_id}_flow'][0])}")
        print(f"  Mean angle: {np.mean(data[f'trial_{first_trial_id}_flow'][1])}")
    
    return data

Saved optical flow data to /Users/ankushdhawan/Documents/Stanford/Pupper/tactile_gait/pupper_tactile_gait/classification/data_processing/dataset/0 degrees_new/0-felt_pad/npz_files/0-felt_pad_flow_data.npz
Total number of trials: 10

Trial 1:
  Optical flow shape: (2, 40, 53)
  Mean magnitude: 0.2973407879888812
  Mean angle: 3.006030562019222

Trial 2:
  Optical flow shape: (2, 40, 53)
  Mean magnitude: 0.440655447652655
  Mean angle: 3.0559763109329032

Trial 3:
  Optical flow shape: (2, 40, 53)
  Mean magnitude: 0.4538839676025556
  Mean angle: 2.877223244819017


In [209]:
# Uncomment to test loading the flow NPZ file
flow_data = load_flow_data(flow_npz_path)

Available data in the NPZ file: ['trial_ids', 'trial_1_flow', 'trial_1_filenames', 'trial_1_shape', 'trial_2_flow', 'trial_2_filenames', 'trial_2_shape', 'trial_3_flow', 'trial_3_filenames', 'trial_3_shape', 'trial_4_flow', 'trial_4_filenames', 'trial_4_shape', 'trial_5_flow', 'trial_5_filenames', 'trial_5_shape', 'trial_6_flow', 'trial_6_filenames', 'trial_6_shape', 'trial_7_flow', 'trial_7_filenames', 'trial_7_shape', 'trial_8_flow', 'trial_8_filenames', 'trial_8_shape', 'trial_9_flow', 'trial_9_filenames', 'trial_9_shape', 'trial_10_flow', 'trial_10_filenames', 'trial_10_shape']
Trial IDs: [ 1  2  3  4  5  6  7  8  9 10]

First trial (ID: 1):
  Flow data shape: (2, 40, 53)
  Mean magnitude: 0.2973407879888812
  Mean angle: 3.006030562019222


# Visualize Flows

In [210]:
# import os
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
# from matplotlib import cm

# def visualize_optical_flow_sequence(image_paths, output_path, filename_prefix="flow_sequence", 
#                                    arrow_spacing=2, arrow_scale=0.5, fps=5):
#     """
#     Visualizes optical flow across a sequence of images over time.
    
#     Args:
#         image_paths (list): List of paths to image files in sequence order
#         output_path (str): Directory to save the video file
#         filename_prefix (str): Prefix for the output filename
#         arrow_spacing (int): Spacing between arrows (higher means fewer arrows)
#         arrow_scale (float): Scale factor for arrow length
#         fps (int): Frames per second for the output video
        
#     Returns:
#         str: Path to the saved video file
#     """
#     # Create output directory if it doesn't exist
#     os.makedirs(output_path, exist_ok=True)
    
#     # Load all frames
#     frames = []
#     for img_path in image_paths:
#         frame = cv2.imread(img_path)
#         if frame is not None:
#             frames.append(frame)
    
#     if len(frames) < 2:
#         print(f"Insufficient frames found ({len(frames)}), need at least 2")
#         return None
    
#     # Get dimensions from first frame
#     first_frame = frames[0]
#     gray_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
#     height, width = gray_frame.shape
    
#     # Create coordinates for the arrows
#     y, x = np.mgrid[0:height:arrow_spacing, 0:width:arrow_spacing]
    
#     # Create figure for visualization
#     fig = plt.figure(figsize=(10, 8))
#     ax = fig.add_subplot(111)
    
#     # Create a list to store all flow data for animation
#     flow_data_list = []
#     prev_gray = gray_frame
    
#     # Calculate optical flow for each consecutive pair of frames
#     for i in range(1, len(frames)):
#         # Convert current frame to grayscale
#         curr_frame = frames[i]
#         curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        
#         # Calculate optical flow
#         flow = cv2.calcOpticalFlowFarneback(
#             prev_gray, curr_gray,
#             None, 0.5, 2, 8, 2, 5, 1.1, 0
#         )
        
#         # Convert to polar coordinates (magnitude and angle)
#         mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
#         # Sample the magnitudes and angles at the arrow grid points
#         mag_subset = mag[::arrow_spacing, ::arrow_spacing]
#         ang_subset = ang[::arrow_spacing, ::arrow_spacing]
        
#         # Convert to cartesian coordinates
#         u = mag_subset * np.cos(ang_subset)
#         v = mag_subset * np.sin(ang_subset)
        
#         # Save flow data for later animation
#         flow_data_list.append((u, v, mag_subset, curr_frame))
        
#         # Update previous frame
#         prev_gray = curr_gray
    
#     # Function to animate the optical flow over time
#     def animate_flow(i):
#         ax.clear()
#         ax.set_xticks([])
#         ax.set_yticks([])
        
#         # Set consistent view
#         ax.set_xlim(0, width)
#         ax.set_ylim(height, 0)  # Invert y-axis to match image coordinates
        
#         # Get flow data for this frame
#         u, v, mag, frame = flow_data_list[i]
        
#         # Remove this line that displays the background image:
#         # ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
#         # Set a white/light background instead
#         ax.set_facecolor('whitesmoke')
        
#         # Plot the flow vectors
#         mag_flat = mag.flatten()
#         norm = plt.Normalize(vmin=0, vmax=np.max(mag_flat) * 0.8)  # Clip max to improve color range
        
#         quiver = ax.quiver(x, y, u, v, mag_flat, cmap='plasma', norm=norm,
#                         scale=1.0/arrow_scale, width=0.003, headwidth=3, headlength=4)
                        
#         ax.set_title(f'Optical Flow: Frame {i+1}/{len(flow_data_list)}')
        
#         return quiver,
    
#     # Create animation
#     video_path = os.path.join(output_path, f"{filename_prefix}_video.mp4")
    
#     # Add colorbar with a fixed position so it doesn't change each frame
#     plt.tight_layout()
#     cb_ax = fig.add_axes([0.9, 0.1, 0.02, 0.8])  # [left, bottom, width, height]
    
#     # Set consistent normalization for the colorbar
#     all_mags = np.concatenate([data[2].flatten() for data in flow_data_list])
#     norm = plt.Normalize(vmin=0, vmax=np.max(all_mags) * 0.8)
#     cb = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='plasma'), cax=cb_ax)
#     cb.set_label('Flow Magnitude')
    
#     # Generate animation
#     num_frames = len(flow_data_list)
#     anim = FuncAnimation(fig, animate_flow, frames=num_frames, 
#                          interval=1000/fps, blit=True)
    
#     # Save animation
#     try:
#         anim.save(video_path, writer='ffmpeg', fps=fps, dpi=150)
#     except Exception as e:
#         print(f"Error saving with ffmpeg: {e}")
#         try:
#             anim.save(video_path, writer='pillow', fps=fps, dpi=150)
#         except Exception as e2:
#             print(f"Error saving with pillow: {e2}")
#             # Save a static image of the first frame
#             animate_flow(0)
#             plt.savefig(os.path.join(output_path, f"{filename_prefix}_static.png"))
    
#     plt.close(fig)
#     print(f"Saved flow sequence visualization to {video_path}")
#     return video_path

# def visualize_all_trial_sequences(trials, images_dir, output_dir):
#     """
#     Visualizes optical flow sequences for all trials.
    
#     Args:
#         trials (dict): Dictionary mapping trial IDs to lists of image filenames
#         images_dir (str): Base directory containing the images
#         output_dir (str): Base directory to save visualizations
    
#     Returns:
#         list: Paths to all generated video files
#     """
#     video_paths = []
    
#     for trial_id, image_files in trials.items():
#         try:
#             # Get full paths to the images
#             image_paths = [os.path.join(images_dir, img) for img in image_files]
            
#             # Create subdirectory for this trial
#             trial_dir = os.path.join(output_dir, f"trial_{trial_id}")
            
#             # Visualize the sequence
#             video_path = visualize_optical_flow_sequence(
#                 image_paths,
#                 trial_dir,
#                 filename_prefix=f"flow_trial_{trial_id}",
#                 arrow_spacing=5,  # Increased spacing for clarity
#                 arrow_scale=0.2   # Stronger scaling for better visibility
#             )
            
#             if video_path:
#                 video_paths.append(video_path)
#                 print(f"Completed visualization for Trial {trial_id}")
#         except Exception as e:
#             print(f"Error visualizing trial {trial_id}: {e}")
    
#     return video_paths

# # Assuming 'trials' is the dictionary from process_trials
# video_paths = visualize_all_trial_sequences(trials, images_dir, flow_output_dir)
# print(f"Generated {len(video_paths)} visualization videos")